In [9]:
import numpy as np
import random
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker, Polyline, AntPath
from ipywidgets import HTML

In [ ]:
cities = [
  {
    "id": 0,
    "name": "Bezannes",
    "coords": (49.22363900671118, 3.9887723157745576)
  },
  {
    "id" : 1,
    "name": "Bétheny",
    "coords": (49.285131827708035, 4.063798713400151)
  },
  {
    "id" : 2,
    "name": "Cernay-les-Reims",
    "coords": (49.2642453710596, 4.1036913180272)
  },
  {
    "id" : 3,
    "name": "Champfleury",
    "coords": (49.19818615812236, 4.019795395571029)
  },
  {
    "id" : 4,
    "name": "Cormontreuil",
    "coords": (49.22264097621803, 4.049551196446118)
  },
  {
    "id" : 5,
    "name": "Cruny",
    "coords": (49.257059457427616, 3.7369142978612735)
  },
  {
    "id" : 6,
    "name": "Epoye",
    "coords": (49.29041023356542, 4.237901634457861)
  },
  {
    "id" : 7,
    "name": "Jonchery-sur-vesle",
    "coords": (49.28752839149448, 3.8121724734582907)
  },
  {
    "id" : 8,
    "name": "Muizon",
    "coords": (49.27682283902432, 3.889787835151326)
  },
  {
    "id" : 9,
    "name": "Nogent-l'Abbesse",
    "coords": (49.25586664716424, 4.155845017046866)
  },
  {
    "id" : 10,
    "name": "Poilly",
    "coords": (49.216141770809536, 3.8203569577722494)
  },
  {
    "id" : 11,
    "name": "Reims",
    "coords": (49.258352984690276, 4.030731630624372)
  },
  {
    "id" : 12,
    "name": "Taissy",
    "coords": (49.21423458122659, 4.093274461155123)
  },
  {
    "id" : 13,
    "name": "Tinqueux",
    "coords": (49.24715898263178, 3.9988975225936736)
  },
  {
    "id" : 14,
    "name": "Witry-les-Reims",
    "coords": (49.2933840208818, 4.125741773971905)
  }
]

In [11]:
distances = np.matrix(
[
  [ 0, 0, 0, 12, 0, 0, 0, 0, 0, 0, 17, 0, 0, 7, 0],
  [ 0, 0, 9, 0, 0, 0, 0, 0, 23, 0, 0, 17, 0, 0, 8],
  [ 0, 9, 0, 0, 12, 0, 0, 0, 0, 7, 0, 15, 0, 0, 7],
  [12, 0, 0, 0, 12, 0, 0, 0, 0, 0, 21, 0, 16, 0, 0],
  [ 0, 0, 12, 12, 0, 0, 0, 0, 0, 0, 0, 15, 9, 14, 0],
  [ 0, 0, 0, 0, 0, 0, 0, 13, 0, 0, 12, 0, 0, 28, 0],
  [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 10],
  [ 0, 0, 0, 0, 0, 13, 0, 0, 10, 0, 0, 0, 0, 0, 0],
  [ 0, 23, 0, 0, 0, 0, 0, 10, 0, 0, 0, 19, 0, 13, 0],
  [ 0, 0, 7, 0, 0, 0, 8, 0, 0, 0, 0, 0, 10, 0, 0],
  [17, 0, 0, 21, 0, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [ 0, 17, 15, 0, 15, 0, 0, 0, 19, 0, 0, 0, 0, 13, 0],
  [ 0, 0, 0, 16, 9, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0],
  [ 7, 0, 0, 0, 14, 28, 0, 0, 13, 0, 0, 13, 0, 0, 0],
  [ 0, 8, 7, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0]
]
)

In [ ]:
def convert_graph(graph):
  graph_dijkstra = Graph()
  for i in range(len(graph)):
    for j in range(len(graph)):
      graph_dijkstra.add_edge(i, j,graph[i][j])
  return graph_dijkstra

def get_distance_between_vertices(graph, i, j):
  if (graph[i][j] != np.inf):
    return graph[i][j], [i, j]
  else:
    if i != j:
      shortest_path = find_path(graph, i,j)
      return shortest_path.total_cost, shortest_path.nodes
    else:
      return 0, None

def get_complete_graph(graph, vertices_list):
  all_paths = {}
  converted_graph = convert_graph(graph)
  complete_graph = np.zeros((len(vertices_list), len(vertices_list)))
  for vertex in vertices_list:
    successors = np.delete(vertices_list, np.where(vertices_list == vertex))
    for successor in successors:
      distance, path = get_distance_between_vertices(converted_graph, vertex, successor)
      i = np.where(vertices_list == vertex)[0][0]
      j = np.where(vertices_list == successor)[0][0]
      complete_graph[i][j] = distance
      all_paths[(i, j)] = path
  return complete_graph, all_paths

In [ ]:
import pickle

data = {
  "graph": distances,
  "complete_graph": complete_graph,
  "all_paths": all_paths,
  "vertices_list": vertices_list,
  "start_vertex": start_vertex,
  "start_vertex_index": start_vertex_index,
  "interval_vertex": interval_vertex,
  "interval_vertex_index": interval_vertex_index,
  "interval": interval,
  "all_vertex": all_vertex
}

with open('data_test.pickle', 'wb') as file:
    pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
def get_random_cities(nb_cities, nb_cities_chosen):
  random_cities = []
  all_cities = np.array(range(nb_cities))
  for _ in range(nb_cities_chosen):
    value = random.choice(all_cities)
    random_cities.append(value)
    all_cities = np.delete(all_cities, np.where(all_cities == value))
  return np.array(random_cities)

In [13]:
def draw_cities(m):
  for city in cities:
    circle_marker = CircleMarker()
    circle_marker.location = city['coords']
    circle_marker.radius = 10
    circle_marker.fill_opacity = 1
    if city["id"] == start_city:
      circle_marker.color = "red"
      circle_marker.fill_color = "red"
    elif city['id'] in cities_chosen:
      circle_marker.color = "green"
      circle_marker.fill_color = "green"
    else:
      circle_marker.color = "gray"
      circle_marker.fill_color = "gray"
    popupMessage = HTML()
    popupMessage.value = city['name']
    circle_marker.popup = popupMessage
    m.add_layer(circle_marker)

def draw_lines(m):
  all_paths = []
  for i in range(distances.shape[0]):
    for j in range(distances.shape[1]):
      if distances[i, j] != 0:
        all_paths.append([cities[i]['coords'], cities[j]['coords']])
  line = Polyline(
      locations=all_paths,
      color="gray",
      fill=False
  )
  m.add_layer(line)

def add_path(m):
  complete_path = []

  for i in range(len(path)):
    complete_path.append(cities[path[i]]['coords'])

  ant_path = AntPath(
    locations=complete_path,
    dash_array=[1, 10],
    delay=1000,
    color='red',
    pulse_color='pink'
  )

  m.add_layer(ant_path)

In [14]:
cities_chosen = get_random_cities(len(cities), 10)
path = np.append(cities_chosen, cities_chosen[0])
start_city = path[0]

In [15]:
m_1 = Map(
  basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
  center=cities[11]['coords'],
  zoom=10.5,
  close_popup_on_click=False
)

draw_lines(m_1)
draw_cities(m_1)

m_1

Map(center=[49.258352984690276, 4.030731630624372], close_popup_on_click=False, controls=(ZoomControl(options=…

In [16]:
m_2 = Map(
  basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
  center=cities[11]['coords'],
  zoom=10.5,
  close_popup_on_click=False
)

draw_lines(m_2)
draw_cities(m_2)
add_path(m_2)

m_2

Map(center=[49.258352984690276, 4.030731630624372], close_popup_on_click=False, controls=(ZoomControl(options=…